In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import pinv, inv, matrix_rank, eigvals, eig
from scipy.linalg import orth
from sklearn.cluster import KMeans

def BKHK2(X):
    p = 2
    d, n = X.shape
    kmeans = KMeans(n_clusters=2, random_state=42).fit(X.T)
    c = kmeans.cluster_centers_.T
    CO = c
    k = n // 2
    E = np.zeros((n, 2))
    epsilon = 1e-6
    e = 1
    while e > epsilon:
        for i in range(n):
            for j in range(2):
                E[i, j] = np.linalg.norm(X[:, i] - CO[:, j]) ** 2
        delta_e = E[:, 0] - E[:, 1]
        idx = np.argsort(delta_e)
        idx1 = idx[:k]
        idx2 = idx[k:]
        g = np.zeros(n)
        g[idx1] = 1
        G = np.column_stack((g, 1 - g))
        XC = [X[:, idx1], X[:, idx2]]
        C1 = np.mean(XC[0], axis=1)
        C2 = np.mean(XC[1], axis=1)
        C = np.column_stack((C1, C2))
        e = np.linalg.norm(CO - C)
        CO = C
    idxt = [idx1.tolist(), idx2.tolist()]
    return XC, idxt, G

def BKHK(X, p):
    d, n = X.shape
    POSO = {1: list(range(n))}
    data = {1: X}
    for i in range(p):
        DATA = {}
        for j in range(1, 2 ** i + 1):
            D = data[j]
            pos = POSO[j]
            XC, idx, _ = BKHK2(D)
            POS = {}
            POS[2 * j - 1] = [pos[i] for i in idx[0]]
            POS[2 * j] = [pos[i] for i in idx[1]]
            DATA[2 * j - 1] = XC[0]
            DATA[2 * j] = XC[1]
        POSO.update(POS)
        data = DATA
    U = np.zeros((d, 2 ** p))
    for k in range(1, 2 ** p + 1):
        U[:, k - 1] = np.mean(X[:, POSO[k]], axis=1)
    return U, POSO

def CLRA(X, U, s):
    d, n = X.shape
    d1, n1 = U.shape
    H = np.zeros((n, n1))
    HS = np.zeros((n, n1))
    POSH = np.zeros((n, n1), dtype=int)
    for i in range(n):
        for j in range(n1):
            H[i, j] = np.linalg.norm(X[:, i] - U[:, j]) ** 2
        HS[i, :] = np.sort(H[i, :])
        POSH[i, :] = np.argsort(H[i, :])
    B = np.zeros((n, n1))
    for i in range(n):
        denom = np.sum(HS[i, s] - HS[i, :s])
        for j in range(n1):
            if j >= s:
                B[i, POSH[i, j]] = 0
            else:
                B[i, POSH[i, j]] = (HS[i, s] - HS[i, j]) / denom
    Delta = np.diag(np.sum(B, axis=0)) + np.finfo(float).eps * np.eye(n1)
    I = np.eye(n)
    A = B @ np.linalg.inv(Delta) @ B.T
    P = B @ np.linalg.inv(np.sqrt(Delta))
    A = (A + A.T) / 2
    L = I - A
    return L, P

def L20_SSD(A, k, W, NITER):
    for _ in range(NITER):
        pinvAW = pinv(W.T @ A @ W)
        P = A @ W @ pinvAW @ W.T @ A
        diagP = np.diag(P)
        index = np.argsort(-diagP)
        indexW = index[:k]
        indexO = index[k:]
        MW = A @ W
        MP = MW[indexW, :]
        OMP = orth(MP)
        d, m = W.shape
        RM = matrix_rank(OMP)
        if RM != m:
            Z = np.zeros((OMP.shape[0], m - RM))
            OMP = np.hstack((OMP, Z))
        W[indexW, :] = OMP
        W[indexO, :] = 0
    Obj1 = 0
    return W, Obj1

def SSDr(M, m, k):
    eigvals_, eigvecs = np.linalg.eigh(M)
    indices = np.argsort(eigvals_)[::-1]
    W = eigvecs[:, indices[:m]]
    return W

def SFESA(L, X, alpha, m, k):
    d, n = X.shape
    I = np.eye(n)
    Ln = alpha * inv(L + alpha * I)
    M = alpha * X @ (Ln - I) @ X.T
    M = (M + M.T) / 2
    em = eigvals(M)
    R = matrix_rank(M)
    eta = np.sort(em)
    if eta[0] < 0:
        M = M - eta[0] * np.eye(d) + np.finfo(float).eps * np.eye(d)
    em = eigvals(M)
    R = matrix_rank(M)
    if R <= m:
        W = SSDr(M, m, k)
    else:
        NITER = 1
        rr = False
        while not rr:
            WO = np.random.rand(d, m)
            if matrix_rank(WO) == m:
                rr = True
            id_zero = np.random.choice(d, size=(d - k), replace=False)
            WO[id_zero, :] = 0
            WO = orth(WO)
            W, _ = L20_SSD(M, k, WO, NITER)
    F = Ln @ X.T @ W
    scores = np.sum(W.T * W.T, axis=1)
    S = np.argsort(scores)[::-1][:m]
    return W, F, S

def demo_SFESA(X, s, p, alpha, m, k):
    U, POS = BKHK(X, p)
    L, _ = CLRA(X, U, s)
    W, F, S = SFESA(L, X, alpha, m, k)
    return W, F, S


In [ ]:


if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\PC\\Desktop\\PFE-MNSA\\DATA\\PCMAC.mat")
    X = df.values.T
    s = 3
    p = 3
    alpha = 0.01
    m = 10
    k = 10
    W, F, S = demo_SFESA(X, s, p, alpha, m, k)
    print("\nMatrice de projection W :\n", W)
    print("\nVecteurs propres F :\n", F)
    print("\nIndices des meilleures features S :\n", S)
